## LTV Audit
    To verify LTV values are consistent based on transaction tables

#### Use these queries to collect data as the input csv files for this notebook to process
    --- prepare data for LTV correction
    --- pre implement to produce ltv0.csv
```
    select count(*) as number_of_bookings,
     round(sum(r.total/(select a2.transaction_attribute_value
     from R2_TransactionAttributes a1, R2_TransactionAttributes a2
     where a1.R2_Transactions_transaction_id =a2.R2_Transactions_transaction_id
     and a1.transaction_attribute_name='conversionCode' and a2.transaction_attribute_name='conversionRate'
     and a1.transaction_attribute_value=i.currency_code limit 1)),2) as ltv0, idMember
     from R2_Itineraries i
    join R2_RoomConfirmations r on r.R2_Itineraries_itinerary_id=i.itinerary_id
     join members m on m.idMembers = i.idMember and m.groups='active'
     left join member_loyalty ml on i.idMember=ml.member_id
     where i.creation_date < '2020-10-15 05:00:00' 
     and (i.STATUS not in ('Cancelled','Cancel') OR i.STATUS is null) and (i.local_status not in ('CANCELLED') OR i.local_status is null)
    and r.local_status!='CANCELLED'
     and r.status not in ('CANCELLED','CANCELED')
    group by idMember
    order by idMember              
```

--- post implement to produce ltv1.csv
```
    select count(*) as number_of_bookings,
     round(sum(r.total/(select a2.transaction_attribute_value
     from R2_TransactionAttributes a1, R2_TransactionAttributes a2
     where a1.R2_Transactions_transaction_id =a2.R2_Transactions_transaction_id
     and a1.transaction_attribute_name='conversionCode' and a2.transaction_attribute_name='conversionRate'
     and a1.transaction_attribute_value=i.currency_code limit 1)),2) as ltv1, idMember
     from R2_Itineraries i
    join R2_RoomConfirmations r on r.R2_Itineraries_itinerary_id=i.itinerary_id
     join members m on m.idMembers = i.idMember and m.groups='active'
     left join member_loyalty ml on i.idMember=ml.member_id
     where i.creation_date >= '2020-10-15 05:00:00' and i.end_date < '2021-03-03 00:00:00' 
     and (i.STATUS not in ('Cancelled','Cancel') OR i.STATUS is null) and (i.local_status not in ('CANCELLED') OR i.local_status is null)
    and r.local_status!='CANCELLED'
     and r.status not in ('CANCELLED','CANCELED')
    group by idMember
    order by idMember
```

-- current loyalty levels to produce ltv.csv
```
    select ml.*, m.groups from member_loyalty ml, members m where ml.member_id=m.idMembers;
```

-- produce ltv_credits.csv
```
    select sum(amount) as credits, member_id 
    from  member_credits 
    where credit_type='LTV_REWARD' and amount>0 
    group by member_id 
    order by member_id;
```

In [ ]:
import numpy as np
import os
import pandas as pd
import re

In [ ]:
base_dir = './preparation/'
ltv_file = base_dir + 'ltv.csv'
ltv0_file = base_dir + 'ltv0.csv'
ltv1_file = base_dir + 'ltv1.csv'
ltv_credits_file = base_dir + 'ltv_credits.csv'

raw_ltv = \
     pd.read_csv(ltv_file, usecols=['member_id', 'hotel_ltv','hotel_ltv_tstamp','hotel_tier_ltv','groups'], \
                 dtype= { "member_id":np.character, "hotel_ltv":np.double, "hotel_ltv_tstamp":np.character, "hotel_tier_ltv":np.double, "groups":np.character}, float_precision='high')
raw_ltv0 = \
     pd.read_csv(ltv0_file, usecols=['number_of_bookings', 'ltv0','idMember'], \
                 dtype= { "number_of_bookings":np.uint8, "ltv0":np.double, "idMember":np.character}, float_precision='high')
raw_ltv0.rename(columns={"number_of_bookings": "bookings0", "idMember": "member_id"}, inplace=True)

raw_ltv1 = \
     pd.read_csv(ltv1_file, usecols=['number_of_bookings', 'ltv1','idMember'], \
                 dtype= { "number_of_bookings":np.uint8, "ltv1":np.double, "idMember":np.character}, float_precision='high')
raw_ltv1.rename(columns={"number_of_bookings": "bookings1","idMember": "member_id"}, inplace=True)

raw_ltv_credits = \
     pd.read_csv(ltv_credits_file, usecols=['member_id', 'credits'], \
                 dtype= { "member_id":np.character, "credits":np.double}, float_precision='high')

In [ ]:
# raw_ltv_credits.head(3)
# raw_ltv0.head(3)
# raw_ltv1.head(3)
raw_ltv.head(3)

In [ ]:
raw_ltv.shape, raw_ltv0.shape, raw_ltv1.shape, raw_ltv_credits.shape

In [ ]:
ltv_0_1=raw_ltv0.merge(raw_ltv1, on='member_id', how='outer')

In [ ]:
ltv_0_1.head()

In [ ]:
ltv_0_1_2=raw_ltv.merge(ltv_0_1, on='member_id', how='left')

In [ ]:
ltv_0_1_2.shape

In [ ]:
raw_ltv_credits.loc[raw_ltv_credits.member_id.isin(['302057','970793'])]

In [ ]:
ltv_all=ltv_0_1_2.merge(raw_ltv_credits, on='member_id', how='left')

In [ ]:
ltv_all.shape

In [ ]:
ltv_all.loc[ltv_all.member_id.isin(['302057','970793'])]

In [ ]:
ltv_all.head()

In [ ]:
ltv_all.fillna({'hotel_ltv': 0,'hotel_tier_ltv': 0,'bookings0': 0,'bookings1': 0, 'credits': 0, 'ltv0': 0, 'ltv1': 0}, inplace=True)
ltv_all['bookings'] = ltv_all['bookings0'] + ltv_all['bookings1']
ltv_all['ltv_0_1'] = ltv_all['ltv0'] + ltv_all['ltv1']
ltv_all['ml_ltv_total'] = ltv_all['hotel_ltv'] + ltv_all['hotel_tier_ltv']
ltv_all['missing_ltv'] = ltv_all['ltv_0_1'] - ltv_all['ml_ltv_total']
ltv_all.sort_values('missing_ltv', ascending=False, inplace=True)

In [ ]:
ltv_all.loc[ltv_all.member_id.isin(['302057','970793','612841','683761'])]

In [ ]:
# raw_ltv.head(3)
# raw_ltv1.head(3)

In [ ]:
ltv_0_1_2.loc[ltv_0_1_2.member_id.isin(['302057','970793','683761'])]

In [ ]:
raw_ltv0[raw_ltv0.member_id.isin(['302057','970793'])]

In [ ]:
raw_ltv1.loc[raw_ltv1.member_id.isin(['302057','970793'])]

In [ ]:
raw_ltv.loc[raw_ltv.member_id.isin(['302057','970793','683761'])]

In [ ]:
ltv_all.to_csv(base_dir + 'ltv_out.csv', \
               columns=["member_id","groups","credits","hotel_ltv_tstamp","bookings0","bookings1","bookings","hotel_ltv","hotel_tier_ltv","ml_ltv_total","ltv0","ltv1","ltv_0_1","missing_ltv"], \
               encoding="utf-8", index=False, float_format='%.2f') 